<a href="https://colab.research.google.com/github/abigailsleek/TeamRuby/blob/master/recommendation_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# section1

import all the required libraries


In [0]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from scipy import stats
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


next, load the data set to pandas data frame

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
posts = pd.read_csv('/content/drive/My Drive/Member/posts.csv')
following =pd.read_csv('/content/drive/My Drive/Member/following.csv')
thoughts = pd.read_csv('/content/drive/My Drive/Member/thoughts.csv')
notifications = pd.read_csv('/content/drive/My Drive/Member/notifications.csv')
extfeeds = pd.read_csv('/content/drive/My Drive/Member/extfeeds.csv')
mailists = pd.read_csv('/content/drive/My Drive/Member/maillists.csv')
migrations = pd.read_csv('/content/drive/My Drive/Member/migrations.csv')
users = pd.read_csv('/content/drive/My Drive/Member/users.csv')
contact_settings = pd.read_csv('/content/drive/My Drive/Member/contact_settings.csv')


In [0]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 12 columns):
id            876 non-null int64
user_id       876 non-null int64
title         876 non-null object
content       876 non-null object
tags          182 non-null object
slug          876 non-null object
created_at    876 non-null object
updated_at    876 non-null object
image         400 non-null object
status_id     0 non-null float64
action        5 non-null object
post_id       0 non-null float64
dtypes: float64(2), int64(2), object(8)
memory usage: 82.2+ KB


In [0]:
posts.shape

(876, 12)

In [0]:
posts.drop(['updated_at','image','action', 'created_at', 'status_id'], axis=1, inplace=True)
posts.head()

,id,user_id,title,content,tags,slug,post_id
0,1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,NaN
1,2,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,NaN
2,4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,NaN
3,6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,NaN
4,7,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,NaN


In [0]:
users.drop(['remember_token','created_at', 'updated_at','password','provider_id','image','provider','username'], axis=1, inplace=True)
users.head()

,id,name,email,short_bio
0,1,Eniayomi Oluwaseyi,oluwaseyieniayomi@gmail.com,Software Developer | DevOPs Engineer
1,2,Elijah Okokon,okoelijah@gmail.com,Web Developer
2,3,Jeffrey Ogah,jeff.ogah@gmail.com,Front End Developer | React Developer | Mentor...
3,4,Oluwaseyi Oluwapelumi,nathanoluwaseyi@gmail.com,| Software Developer | DevOps Engineer | @linu...
4,5,PoRH,paulchibiukeigweze@gmail.com,I Am lamar and you don't think am real?


so i want to clean out my posts data 

In [0]:

posts['content'] = posts['content'].str.replace(r'<[^>]*>', '')
posts['content'] = posts['content'].str.replace(r'\s', ' ')
posts['content'] = posts['content'].str.replace(r'\\', ' ')
posts['content'] = posts['content'].str.replace(r'\~', ' ')
posts['content'] = posts['content'].str.replace(r'\[.*?\]', '')
posts['content'] = posts['content'].str.replace(r'\(.*?\)', '')
posts['content'].head()

0    I learnt how to use the table tag as i have us...
1     I am on this journey with start.ng, and here ...
2    I have not been attending classes on the HNG c...
3    My journey on **StartNG** pre-internship progr...
4     A Summary on The “idongesit.html” CV, Its Str...
Name: content, dtype: object

#Section 2

so i'm going to be applying some algorithms 

In [0]:

tfidf_mypost = TfidfVectorizer(stop_words='english')

posts['content'] = posts['content'].fillna('')

tfidf_mypost_mat = tfidf_mypost.fit_transform(posts['content'])

tfidf_mypost_mat.shape

(876, 4597)

In [0]:

cos_sim = linear_kernel(tfidf_mypost_matrix, tfidf_mypost_matrix)


In [0]:

indices = pd.Series(posts.index, index=posts['title']).drop_duplicates()

i'm going to define a function that gets posts by title.

In [0]:

def get_recommended_posts(title, cos_sim=cos_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    post_indices = [i[0] for i in sim_scores]
    return posts['title'].iloc[post_indices]

In [0]:
get_recommended_posts('HTML BEGINS HERE')

99                     Introduction To HTML
43         HTML Course so far on startdotng
487                  First Class About HTML
6                   StartNG HTML Exposition
26                       The Journey so far
646    How To Create A simple CV Using HTML
647    How To Create A simple CV Using HTML
92                               Hello HTML
396                  What I've done so far.
397                  What I've done so far.
Name: title, dtype: object

you could test this similarity post recommender by running :

In [0]:
get_recommended_posts('input the post title here please')

so i'm going to be doing something similar for "who to follow"

In [0]:
users['short_bio'].head(10)

0                 Software Developer | DevOPs Engineer
1                                        Web Developer
2    Front End Developer | React Developer | Mentor...
3    | Software Developer | DevOps Engineer | @linu...
4              I Am lamar and you don't think am real?
5                                        My Open Diary
6                                          An engineer
7    Software Developer hotelsng, Ex. Frontend deve...
8    Front-end Engineer 💻 | FIFA advocate 🎮 |  Ninja 🥋
9                                                 Here
Name: short_bio, dtype: object

applying the same algorithm as before:

In [0]:

tfidf_myusers = TfidfVectorizer(stop_words='english')

users['short_bio'] = users['short_bio'].fillna('')

tfidf_myusers_mat = tfidf_myusers.fit_transform(users['short_bio'])

tfidf_myusers_mat.shape

(2293, 2242)

In [0]:

cos_sim = linear_kernel(tfidf_myusers_mat, tfidf_myusers_mat, True)

indices = pd.Series(users.index, index=users['id'])

so i  want to create a function that basically transforms data it will get from my get_new_followers, into a new database

In [0]:
def create_my_new_db(x):

    y = x.copy()
    z = x.copy()
    y.update(users['name'])
    z.update(users['short_bio'])
    xdf=pd.DataFrame(x)
    ydf=pd.DataFrame(y)
    zdf=pd.DataFrame(z)
    ydf.rename(columns={'id':'Name'}, inplace=True)
    xdf.rename(columns={'id':'User_Id'}, inplace=True)
    zdf.rename(columns={'id':'short_bio'}, inplace=True)
    frames = [xdf,ydf,zdf]
    new_df = pd.concat(frames, axis=1)
    new_df.set_index('User_Id', inplace=True)
    return new_df

i'm going to just define a function that gets new followers  based on their id.

In [0]:

def get_new_followers(id, cos_sim=cos_sim):
    idx = indices[id]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    follow_indices = [i[0] for i in sim_scores]
    follower_id = users['id'].iloc[follow_indices]
     
    new_record = create_my_new_db(follower_id)
    return new_record

so i'm going to create a function just for viewing my user via id

In [0]:

def view_user(id):
    name = users.loc[users['id'] == id, 'name']
    bio = users.loc[users['id'] == id, 'short_bio']
    return name.iloc[0], bio.iloc[0]

In [0]:
view_user(66)

('Kev Chike', 'Web Developer')

In [0]:
users.loc[users['id'] == 66, 'short_bio'].iloc[0]

'Web Developer'

#  Section 3

using the KNN alogrithm